<a href="https://colab.research.google.com/github/HkgCory/Tensorflow-deeplearning-colab-dog_breed/blob/main/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# !unzip '/content/drive/MyDrive/deepLearning-dog/dog-breed-identification.zip' -d '/content/drive/MyDrive/deepLearning-dog/'

# end to end multiclass dog breed classification

This notebook builds an end to end multi class image classification using tensorflow 2.0 and tensorflow hub

#1 problem
identifying the breed of a dog given an image of a dog.

When Im sitting at the ccafe and i take a photo of a dog, I want to know what breed of a dog it is

#2 data.

The data we're using is from Kaggle's dog breed identification competition. 

# 3. Evaluation

The evaluation is a file with prediction probabilities for each dog breed of each test images.

#4. features

some information about the data:

we'r dealing with images(unstructured data) so it's probably best we use deep learning / transfer learning. 

there are 120 breeds of dogs, this means there are 120 different classes.

There are around 10000+ images in the training set.(these images have labels)

There are around 10000+ images in the test set( these images have no labels, because we will want to predict them.


#Get our workspace ready



In [4]:
import tensorflow as tf
print('TF version: ', tf.__version__)
import tensorflow_hub as hub
print('TF Hubversion: ', hub.__version__)
# check for gpu
print('gpu', 'available (yes!)' if tf.config.list_physical_devices('GPU') else 'not available')

TF version:  2.11.0
TF Hubversion:  0.12.0
gpu not available


# Get our Data Ready

With all machine learning models, our data has to be innumerical format. so we will have to convert the data first, turning the images into tensors (numerical representation). command m h to open keyboard preferences.

In [5]:
# checkout labels of our data

labels_csv = pd.read_csv('/content/drive/MyDrive/deepLearning-dog/labels.csv')
# print(labels_csv.describe())
# print(labels_csv.head())

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#how many images are there oof each breed?

labels_csv['breed'].value_counts().plot.bar(figsize=(20,10))

In [ ]:
labels_csv['breed'].value_counts().median()

In [ ]:
# view image
from IPython.display import Image
Image('/content/drive/MyDrive/deepLearning-dog/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg')

### Getting images and their labels

lets get a list of all our image file pathname


In [ ]:
labels_csv.head()

In [ ]:
# create pathnames from image ID
filenames = ['/content/drive/MyDrive/deepLearning-dog/train/' + fname + '.jpg'for fname in labels_csv['id']]
filenames[:10]

In [ ]:
# check whether number of filenames amtches number of actual image files

import os
if len(os.listdir('/content/drive/MyDrive/deepLearning-dog/train')) == len(filenames):
  print('Filenames match actual amount of files!!! Proceed.')
else:
  print('Filenames do not match actual amount of files, check the target directory.')

In [ ]:
Image(filenames[9000])

In [ ]:
labels_csv['breed'][9000]

now we have our training image filepath in a list, lets prepare labels


In [ ]:
labels = np.array(labels_csv['breed'])
labels


In [ ]:
len(labels)

In [ ]:
#see if number of labels matches the nu,ber of filenames
if len(labels) == len(filenames):
  print('Number of labels amtches number of filenames!')
else:
  print('it doesnt match')

In [ ]:
#find the unique label value
unique_breeds = np.unique(labels)
len(unique_breeds)

In [ ]:
#Turn a single label into an array of booleans
print(labels[0])
labels[0] == unique_breeds

In [ ]:
#turn every label into a boolean array
boolean_labels = [label == unique_breeds for label in labels]
boolean_labels

In [ ]:
len(boolean_labels)

In [ ]:
# example: turning boolean array into integers

print(labels[0])
print(np.where(unique_breeds == labels[0]))
print(boolean_labels[0].argmax())
print(boolean_labels[0].astype(int))

#Creating our own validation set

Since the dataset from kaggle doesnt include validation set, we are going to create our own validation set.

In [ ]:
# Setup X & y variables
X = filenames
y = boolean_labels

In [ ]:
len(filenames)

In [ ]:
#We're going to start off experimenting with 1000 iamges and increase as needed.


In [ ]:
# set number of images to use for experimenting 
NUM_IMAGES = 1000  #@param {type:'slider', min:1000, max:10000, steps:1000}

In [ ]:
#lets split our data into train and validation sets

from sklearn.model_selection import train_test_split
# split them into training and validation of total size NUM_IMAGES

X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES],
                                                  y[:NUM_IMAGES],
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

In [ ]:
# let's have a geez at the training data
X_train[:5], y_train[:2]

#Preprocessing Images (turning images into Tensors)

To preprocess our images into tensors we are going to write a function which does a few things:

1. take an image filepath as input
2. use tensorflow to read the file and save it to a variable, `image`
3. Turn our `image` (a jpg) into Tensors
4. Normalize our image (convert color channel values from 0-255 to 0-1).
5. Resize the `image` to be a shape of(224, 224)
6. Return the modified `image`

In [ ]:
# convert image to numpy array 

from matplotlib.pyplot import imread
image = imread(filenames[42])
image.shape

In [ ]:
image.max(), image.min()

In [ ]:
#turn images into tensors
tf.constant(image)[:3]


In [ ]:
tf.constant(imread(filenames[9000]))[:2]

In [ ]:
#define img size
IMG_SIZE  = 224
#create a function for preprocessing images

def process_image(image_path, img_size = IMG_SIZE):

  #take an image file path and turns the image into a tensor

#read an image file
  image = tf.io.read_file(image_path)
# turn the image into numerical tensor with 3 color channels (RGB)
  image = tf.image.decode_jpeg(image, channels = 3)
# Convert the color channel values from 0-255 to 0-1 value
  image = tf.image.convert_image_dtype(image, tf.float32)
# Resize the iamge to our desired value (224, 224)
  image = tf.image.resize(image, size = [IMG_SIZE, IMG_SIZE])

  return image

In [ ]:
tensor = tf.io.read_file(filenames[26])
tensor

In [ ]:
tf.image.decode_jpeg(tensor, channels = 3)

## Turning our data into batches

Why turn our data into batches???



let's say you're trying to process 10k+ images in one go... they all might not fit into memory.

So that's why we do about 32 (this is batch size) images at a time ( you can manually adjust the batch size if need to be)

In order to use TensorFlow effectively, we need our data in the form of Tensor tuples which look like this:
`(image, label).



In [ ]:
# create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
    Takes an image file path name and the associated label,
    processes the image and returns a tuple of (image,label).
  """
  image = process_image(image_path)
  return image,label

In [ ]:
# demo of the above
(process_image(X[42],y[42]))[:2]

Now we've got a way to turn our data into tuples of Tensors in the form: `(image,label)`, let's make a function to turn all of our data (X & y) into batches

In [ ]:
#define the batch size, 32 is a good start

BATCH_SIZE = 32

#create a function to turn data into batches

def create_data_batches(X,y=None, batch_size=BATCH_SIZE, valid_data = False, test_data=False):
  """
  Creates batches of data out of image(X) and label (y) pairs.
  Shuffles the data if its training data but doesnt shuffle if its validation data.
  Also accepts test data as input (no labels).
    """

    # If the data is a test dataset, we probably don't have labels
  if test_data:
    print('Creating test data batches...')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) #only filepaths (no labels)
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

    #if the data is a valid dataset, we dont need to shuffle it 

  elif valid_data:
    print('Creating validation data batches...')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    print('Creating training data batches')
    # turn filepaths and labels into tensors

    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))

    # shuffling pathnames and labels before mapping image processor function is faster than shuffling
    data = data.shuffle(buffer_size = len(X))
    #create (image, label)tuples (this also turns the image path into a preprocess image)
    data = data.map(get_image_label)
    #Turn the training data into batches
    data_batch = data.batch(BATCH_SIZE)

  return data_batch











In [ ]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
#Checking out the different attributes of our data batches
train_data.element_spec, val_data.element_spec

In [ ]:
y[0]

## Visualizing data batches

our data is now in batches, however, these can be a little hard to understand/comprehend, let's visualize the data batch

In [ ]:
import matplotlib.pyplot as plt

# Create a function for viewing images in a data batch
def show_25_images(images, labels):
  """
  Displays a plot of 25 iamges and their labels from a data batch.
  """

  #setup the figure
  plt.figure(figsize=(10,10))
  #loop through 25(for displaying 25 images)
  for i in range(25):
    #Create subplots (5 rows, 5 columns)
    ax = plt.subplot(5,5, i+1)
    #Display an image
    plt.imshow(images[i])
    # Add the iamge label as the title
    plt.title(unique_breeds[labels[i].argmax()])
    # Turn the grid lines off
    plt.axis('off')

In [ ]:
train_data

In [ ]:
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
unique_breeds[y[0].argmax()]

In [ ]:
unique_breeds[train_labels[5].argmax()]

In [ ]:
# Now let's visualize the data in a training batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
# Now let's visualize our validation set
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

# building a model

before we build a model, there are a few things we need to define:
* the input shape (our images shape, in the form of Tensors) to our model.
* The output shape (image labels, in the form of Tensors) of our model
* The URL of the model we want to use

In [ ]:
# Setup input shape to the model

INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] #batch, height, width, color channels

# setup output shape of our model
OUTPUT_SHAPE = len(unique_breeds)

# setup model URL from TensorFlow Hub

MODEL_URL = 